In [10]:
!pip install spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 72.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
from google.colab import files
uploaded = files.upload()

Saving clean_data_38K.xlsx to clean_data_38K.xlsx


In [6]:
# Read the uploaded Excel file
import pandas as pd
df = pd.read_excel('clean_data_38K.xlsx', sheet_name='Sheet1')

# Load the pre-trained spaCy model for NER
import spacy
nlp = spacy.load("en_core_web_sm")

# Define the function to extract PERSON and ORG entities
def extract_entities(text):
    doc = nlp(text)
    entities = {"persons": [], "organizations": []}

    for ent in doc.ents:
        if ent.label_ == "PERSON":
            entities["persons"].append(ent.text)
        elif ent.label_ == "ORG":
            entities["organizations"].append(ent.text)

    return entities

# Apply the function to extract entities from the 'content' column
df['entities'] = df['content'].apply(extract_entities)

# Show the first few rows with detected entities
df[['content', 'entities']].head()

,content,entities
0,Masks can be a challenge for people w breathle...,"{'persons': ['Brian'], 'organizations': ['Long..."
1,@RNAppleby @KashPrime Take it easy - long Covi...,"{'persons': ['Covid'], 'organizations': []}"
2,#LongCovid &amp; the brain: UC San Francisco r...,"{'persons': ['Covid'], 'organizations': []}"
3,Lung abnormalities found in long COVID patient...,"{'persons': [], 'organizations': []}"
4,@brianvastag The Bell’s disability scale is a ...,"{'persons': [], 'organizations': ['Bell', 'MEC..."


In [13]:
# Extract persons and organizations into separate lists
persons_list = []
organizations_list = []

# Iterate through each row in the 'entities' column
for entity in df['entities']:
    # Directly treat the entity as a dictionary (no need for eval)
    entities = entity  # Assuming 'entity' is already a dictionary with 'persons' and 'organizations' keys

    # Extend the lists with persons and organizations
    persons_list.extend(entities['persons'])
    organizations_list.extend(entities['organizations'])

# Convert the lists to DataFrames
persons_df = pd.DataFrame(persons_list, columns=['Person'])
organizations_df = pd.DataFrame(organizations_list, columns=['Organization'])

In [16]:
persons_df = pd.DataFrame(persons_list, columns=['Person'])
persons_df

,Person
0,Brian
1,Covid
2,Covid
3,Covid
4,Murdoch
...,...
18995,Covid
18996,Covid
18997,Paxlovid
18998,Jonathan


In [17]:
organizations_df

,Organization
0,LongCovid
1,N95
2,Bell
3,MECFS
4,https://t.co/rJplc1nGfk
...,...
23041,LongCovid
23042,London &
23043,LongCovid
23044,Dearest Companions


In [19]:
# Save the organizations and persons DataFrame to a CSV file
organizations_df.to_csv('organizations.csv', index=False)
persons_df.to_csv('persons.csv', index=False)